In [ ]:
from google.colab import drive
drive.mount('alexnet')

In [ ]:
# Alexnet 구현
import torch
import torch.nn as nn
from torchvision import datasets, transforms

devices = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(devices)

In [ ]:
# transforms : 이미지 데이터를 로딩할 때 모듈의 입력값으로 사용할 수 있도록 변환한다.
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(227),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ])

In [ ]:
# Trans and Test set 로딩
# 데이터로 CIFAR-10 사용

trainset = datasets.CIFAR10('~/.data', download=True, train=True, transform =transform)
trainset = torch.utils.data.DataLoader(trainset, batch_size = 256, shuffle = True)

testset = datasets.CIFAR10('~/.data', download = True, train = False, transform = transform)
testset = torch.utils.data.DataLoader(testset, batch_size = 256, shuffle = True)

In [ ]:
# 모델 구축
class AlexNet(nn.Module):
  def __init__(self, input_size = 227, num_classes = 10):
    super(AlexNet, self).__init__()

    self.cnnLayer = nn.Sequential(
        
        # 1st Conv : conv, relu, lrm, pool
        # in_channels = 입력 채널 수, out_channels = 출력 채널 수, kernel_size = 필터 크기
        nn.Conv2d(in_channels = 3, out_channels = 96, kernel_size = 11, padding = 0, stride = 4),
        nn.ReLU(inplace = True),
        nn.LocalResponseNorm(size = 5, alpha = 0.0001, beta = 0.75, k=2), 
        nn.MaxPool2d(kernel_size = 3, stride = 2), # 55 -> 27

        # 2st Conv : conv, relu, lrm, pool

        nn.Conv2d(in_channels = 96, out_channels = 256, kernel_size = 5, padding = 2, stride = 1),
        nn.ReLU(inplace = True),
        nn.LocalResponseNorm(size = 5, alpha = 0.0001, beta = 0.75, k = 2),
        nn.MaxPool2d(kernel_size = 3, stride = 2),

        # 3st Conv : conv, relu

        nn.Conv2d(in_channels = 256, out_channels = 384, kernel_size = 3, padding = 1, stride = 1),
        nn.ReLU(inplace = True),

        # 4st Conv : cont, relu

        nn.Conv2d(in_channels = 384, out_channels = 384, kernel_size = 3, padding = 1, stride = 1),
        nn.ReLU(inplace = True),

        # 5th Conv : conv, relu, lrm, pool
        nn.Conv2d(in_channels = 384, out_channels = 256, kernel_size = 3, padding = 1 , stride = 1),
        nn.ReLU(inplace = True),
        nn.LocalResponseNorm(size = 5, alpha = 0.0001, beta = 0.75, k = 2),
        nn.MaxPool2d(kernel_size = 3, stride = 2),

    )

    self.fcLayer = nn.Sequential(
        # FC Layer
        nn.Linear(6*6*256, 4096),
        nn.ReLU(inplace = True),
        nn.Dropout(p = 0.5),

        nn.Linear(4096, 4096),
        nn.ReLU(inplace = True),
        nn.Dropout(p = 0.5),
        nn.Linear(4096, num_classes)

    )
    # 표준화 및 bias 초기화
    for layer in self.cnnLayer:
      if isinstance(layer, nn.Conv2d) :
        # conv레이어들을 가우시안 분포로 표준화 하고 bias는 0
        nn.init.normal_(layer.weight, mean = 0, std = 0.01)
        nn.init.constant_(layer.bias, 0)

    # 그런데 2,4,5 conv 는 1로 초기화
    nn.init.constant_(self.cnnLayer[4].bias, 1)
    nn.init.constant_(self.cnnLayer[10].bias, 1)
    nn.init.constant_(self.cnnLayer[12].bias, 1)
    
  def forward(self, train):
    # 멀티 GPU는 당장 불가능하기때문에 제외
    output = self.cnnLayer(train)
    output = output.view(-1, 256*6*6)
    output = self.fcLayer(output)

    return output

alexnet = AlexNet(277,10)
alexnet.to(devices)

# weight decay = 0.0005, momentum = 0.9, lr = 0.01
optimizer = torch.optim.SGD(alexnet.parameters(), lr = 0.01, momentum = 0.9, weight_decay = 0.0005)
criterion = nn.CrossEntropyLoss().to(devices)



In [ ]:
# 학습
from tqdm.notebook import tqdm
epochs = 5
for epoch in range(epochs):
  epoch_loss = 0
  for data, classes in tqdm(trainset) :
    inputs, labels = data.to(devices), classes.to(devices)

    optimizer.zero_grad()
    outputs = alexnet(inputs)

    # 순전파, 역전파 최적화
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()

  correct = list(0. for i in range(1000))
  total = list(0. for i in range(1000))

  with torch.no_grad():
    for data, classes in testset:
      inputs, labels = data.to(devices), classes.to(devices)
      outputs = alexnet(inputs)
      _, predicted = torch.max(outputs, 1)
      c = (predicted == labels).squeeze()
      for i in range(labels.size()[0]):
        label = labels[i]
        correct[label] += c[i].item()
        total[label] += 1


  print('{0} : loss {1:.3f}, val_acc {2:.3f}'.format(epoch+1, epoch_loss, (sum(correct) / sum(total))))